In [1]:
import pandas as pd
import os
import json
import numpy as np
import re

In [2]:
## reading the Survey File
#surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\Test.csv",sep="," , skiprows=1)
surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\AmenityClubSurvey FINAL v2 Season Cleaned_deletedFirstTwoColumn.csv",sep="," , skiprows=1, keep_default_na=False)
surveyData.head()

## Reading the Config File and Extracting the Information from there
config_file = open("C:\Projects\MSS\Loading _AmenityData_from_Survey\Python_Codes\ClubCorp\GolfEvents\GolfEventsColMapping.json")
FitnessCenter_config = json.load(config_file)

columnName = ""

In [3]:
def determineBooleanValues (dataFrame):
    val = False
    if pd.notnull(dataFrame[columnName]) :
        if dataFrame[columnName] == columnName:
            val = True
    return val

In [4]:
## Getting the output File name
output_fileName = FitnessCenter_config['outputFileName']


all_column_names = []
FitnessCenter_related_column_names = []

columnIndexForFitnessCenter = []

common_column_count = len(FitnessCenter_config['CommonColumn'])

for i in range(0, common_column_count):
    columnIndexForFitnessCenter.append(FitnessCenter_config['CommonColumn'][i]['columnIndex'])
    all_column_names.append(FitnessCenter_config['CommonColumn'][i]['outputColumnName'])

FitnessCenter_column_count = len(FitnessCenter_config['golfEventsRelatedColumn'])

for j in range(0, FitnessCenter_column_count):
    columnIndexForFitnessCenter.append(FitnessCenter_config['golfEventsRelatedColumn'][j]['columnIndex'])
    all_column_names.append(FitnessCenter_config['golfEventsRelatedColumn'][j]['outputColumnName'])
    FitnessCenter_related_column_names.append(FitnessCenter_config['golfEventsRelatedColumn'][j]['outputColumnName'])

## Data for Fitness Center
FitnessCenter = surveyData.iloc[:, columnIndexForFitnessCenter]
FitnessCenter.columns = all_column_names


filterCondition  = FitnessCenter['ConditionColumnForGolfEvents'] != 'N/A'

clubsNotHavingFitnessCenter = FitnessCenter[~filterCondition]

clubsNotHavingFitnessCenter.to_csv("C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/GolfEvents/GolfEvents_ClubsNotHaving.csv")

FitnessCenter = FitnessCenter[filterCondition]
FitnessCenter['Index']=1
FitnessCenter['Status'] = 'Active'
FitnessCenter['Type'] = 'Tournaments & Outings'
FitnessCenter['Name'] = 'Golf Outings'

for x  in FitnessCenter_related_column_names:
    columnName = x
    FitnessCenter[columnName + '_Boolean'] = FitnessCenter.apply(determineBooleanValues, axis=1)


FitnessCenter.to_csv("C:\\Projects\\MSS\\Loading _AmenityData_from_Survey\\Python_Codes\\ClubCorp\\GolfEvents\\" + output_fileName)